In [ ]:
RNN and LSTM Take home Assignment

In [ ]:
You are provided with a large number of Wikipedia comments which have been labeled by human raters for toxic behavior. The types of toxicity are:

toxic
severe_toxic
obscene
threat
insult
identity_hate

You must create a model which predicts a probability of each type of toxicity for each comment.

Columns
id
toxic
severe_toxic
obscene
threat
insult
identity_hate

In [ ]:
! wget https://videokenoffshore.blob.core.windows.net/videoken-public-data/train.csv

In [5]:
!pip install nltk


  Using cached https://files.pythonhosted.org/packages/c5/10/369f50bcd4621b263927b0a1519987a04383d4a98fb10438042ad410cf88/singledispatch-3.4.0.3-py2.py3-none-any.whl
You are using pip version 9.0.3, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
import pandas as pd
df=pd.read_csv('train.csv')

In [8]:
len(df)

159571

In [9]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
unique_words=set([ word.lower() for line in df['comment_text'] for word in word_tokenize(line)]) 

In [11]:
word2id={}
id2word={}
word2id["__PADDING__"] = 0
word2id["__START__"] = 1
word2id["__UNK__"] = 2
id2word[0]="__PADDING__"
id2word[1]="__START__"
id2word[2]="__UNK__"
for i, word in enumerate(unique_words):
    word2id[word]=i+3
    id2word[i+3]=word

In [26]:
import numpy as np
x,y=[],[]
xv,yv=[],[]

for i,row in df.iterrows():
    if i<5000:
        x.append([word2id[word.lower()] for word in word_tokenize(row['comment_text'])])
        not_toxic=1 if row['toxic']==0 and row['severe_toxic']==0  and row['obscene'] ==0 and row['threat'] ==0 and row['insult'] ==0  and row['identity_hate']==0 else 0 
        y.append([row['toxic'],row['severe_toxic'],row['obscene'],row['threat'],row['insult'],row['identity_hate'],not_toxic])
    elif i>=5000 and i<=8000:
        xv.append([word2id[word.lower()] for word in word_tokenize(row['comment_text'])])
        not_toxic=1 if row['toxic']==0 and row['severe_toxic']==0  and row['obscene'] ==0 and row['threat'] ==0 and row['insult'] ==0  and row['identity_hate']==0 else 0 
        yv.append([row['toxic'],row['severe_toxic'],row['obscene'],row['threat'],row['insult'],row['identity_hate'],not_toxic])
    else:
        break
        


In [27]:
y_train=np.array(y)
X_train=np.array(x)


In [34]:
y_test=np.array(yv)
X_test=np.array(xv)

In [16]:
print('Maximum review length: {}'.format(
len((max((X_train ), key=len)))))

Maximum review length: 2086


In [29]:
from keras.preprocessing import sequence
max_words = 1500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
modelCNN = None
modelRNN=None
modelLSTM=None


In [18]:
vocabulary_size = len(unique_words)

In [19]:
batch_size = 16
num_epochs = 3
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]

In [20]:
from keras.layers import SimpleRNN

In [21]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout,Layer

In [22]:
vocabulary_size = len(unique_words)+10

In [36]:
embedding_size=32
modelLSTM=Sequential()
modelLSTM.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
modelLSTM.add(LSTM(150, dropout=0.2)) 
modelLSTM.add(Dense(7, activation='softmax'))
print(modelLSTM.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1500, 32)          8290848   
_________________________________________________________________
lstm_2 (LSTM)                (None, 150)               109800    
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 1057      
Total params: 8,401,705
Trainable params: 8,401,705
Non-trainable params: 0
_________________________________________________________________
None


In [37]:
modelLSTM.compile(loss='categorical_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [38]:
modelLSTM.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

Train on 4984 samples, validate on 16 samples
Epoch 1/3
4984/4984 [==============================] - 1497s 300ms/step - loss: 0.9111 - acc: 0.8898 - val_loss: 1.1213 - val_acc: 0.8750
Epoch 2/3
4984/4984 [==============================] - 1499s 301ms/step - loss: 0.5924 - acc: 0.9254 - val_loss: 0.7682 - val_acc: 0.9375
Epoch 3/3
4984/4984 [==============================] - 1501s 301ms/step - loss: 0.4402 - acc: 0.9545 - val_loss: 1.0880 - val_acc: 0.8125


In [ ]:
#cant complete the execution .......i m running out of time;(

In [39]:
scores = modelLSTM.evaluate(X_test, y_test, verbose=0)


ValueError: Error when checking input: expected embedding_2_input to have shape (1500,) but got array with shape (1,)

In [33]:
type(modelLSTM)

NoneType